# AI 모의 면접 봇

In [ ]:
import os
import pyttsx3
import speech_recognition as sr
from dotenv import load_dotenv
from openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

engine = pyttsx3.init()
engine.setProperty('rate', 150)  
engine.setProperty('volume', 1)  

applicant = {
    "name": "홍길동",
    "age": 28,
    "degree": "컴퓨터공학 학사",
    "experience": "3년",
    "portfolio": ["웹 개발 프로젝트", "AI 챗봇 개발", "모바일 앱 개발"],
    "skills": ["Python", "Django", "TensorFlow", "React"]
}

def generate_feedback_and_evaluation_with_langchain(answer, question):
    feedback_template = """
    다음 질문에 대한 지원자의 답변에 대한 피드백을 작성해주세요. 질문: {question}, 답변: {answer}
    """
    evaluation_template = """
    다음 질문에 대한 지원자의 답변을 평가해주세요. 질문: {question}, 답변: {answer}. 평가 기준은 직무 관련성, 창의성, 문제 해결 능력 등을 고려해주세요.
    """

    feedback_prompt = PromptTemplate(input_variables=["question", "answer"], template=feedback_template)
    evaluation_prompt = PromptTemplate(input_variables=["question", "answer"], template=evaluation_template)

    chat_model = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))

    feedback_chain = LLMChain(llm=chat_model, prompt=feedback_prompt)
    evaluation_chain = LLMChain(llm=chat_model, prompt=evaluation_prompt)

    feedback = feedback_chain.run({"question": question, "answer": answer})
    evaluation = evaluation_chain.run({"question": question, "answer": answer})

    return feedback, evaluation


def speak(text):
    engine.say(text)
    engine.runAndWait()

def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("음성 입력을 기다리고 있습니다...")
        recognizer.adjust_for_ambient_noise(source)  
        audio = recognizer.listen(source)
    
    try:
        print("음성을 인식 중...")
        return recognizer.recognize_google(audio, language="ko-KR") 
    except sr.UnknownValueError:
        speak("음성을 이해할 수 없습니다. 다시 말해주세요.")
        return None
    except sr.RequestError:
        speak("음성 인식 서비스를 사용할 수 없습니다. 인터넷 연결을 확인해주세요.")
        return None

def generate_question(context):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "system",
            "content": "당신은 기술 회사의 AI 면접관입니다. 한국어로 대화해주세요. 한 번에 하나의 질문만 생성해주세요."
        },
        {
            "role": "user",
            "content": f"다음 맥락을 바탕으로 관련된 면접 질문을 하나만 생성해주세요: {context}"
        }]
    )
    return response.choices[0].message.content

def generate_follow_up_question(last_answer):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "system",
            "content": "당신은 기술 회사의 AI 면접관입니다. 지원자의 답변에 따라 적절한 꼬리 질문을 생성해주세요."
        },
        {
            "role": "user",
            "content": f"다음 답변을 바탕으로 적절한 꼬리 질문을 생성해주세요: {last_answer}"
        }]
    )
    return response.choices[0].message.content

def start_interview():
    print(f"지원자: {applicant['name']}, 나이: {applicant['age']}, 학력: {applicant['degree']}, 경력: {applicant['experience']}\n")
    print("지원자의 포트폴리오:", ', '.join(applicant['portfolio']))
    print("지원자의 주요 기술:", ', '.join(applicant['skills']), "\n")

    speak("📝 모의 면접을 시작합니다! 종료하려면 '종료'라고 말해주세요.\n")

    context = f"지원자 정보: {applicant}"
    question_count = 0
    last_answer = ""
    

    while question_count < 5:  
        if question_count == 0:
            question = generate_question(context)
        else:
            question = generate_follow_up_question(last_answer)

        print(f"면접관: {question}")
        speak(f"면접관: {question}")

        answer = listen()
        if answer is None:
            continue

        print(f"지원자: {answer}")
        if '종료' in answer:
            speak("면접을 종료합니다.")
            print("면접을 종료합니다.")
            break

        context += f"\n질문: {question}\n답변: {answer}"
        last_answer = answer  

        question_count += 1

    speak("모든 질문이 끝났습니다. 이제 피드백과 평가를 제공하겠습니다.")
    feedback, evaluation = generate_feedback_and_evaluation_with_langchain(last_answer, question)
    
    print(f"면접관의 피드백: {feedback}")
    speak(f"면접관의 피드백: {feedback}")
    print(f"면접관의 평가: {evaluation}")
    speak(f"면접관의 평가: {evaluation}")

# 면접 시작
start_interview()



지원자: 홍길동, 나이: 28, 학력: 컴퓨터공학 학사, 경력: 3년

지원자의 포트폴리오: 웹 개발 프로젝트, AI 챗봇 개발, 모바일 앱 개발
지원자의 주요 기술: Python, Django, TensorFlow, React 

면접관: 홍길동 지원자의 프로젝트 경험과 기술들을 고려할 때, 어떤 프로젝트에서 가장 큰 성취감을 느꼈고, 그 이유는 무엇인가요?
음성 입력을 기다리고 있습니다...
음성을 인식 중...
지원자: 모르겠습니다
면접관: 무엇에 대해 모르는 지 자세히 알려주실 수 있나요?
음성 입력을 기다리고 있습니다...
음성을 인식 중...
지원자: 너 공부해 보겠습니다


C:\Users\USER\AppData\Local\Temp\ipykernel_14340\981086241.py:43: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
C:\Users\USER\AppData\Local\Temp\ipykernel_14340\981086241.py:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  feedback_chain = LLMChain(llm=chat_model, prompt=feedback_prompt)
C:\Users\USER\AppData\Local\Temp\ipykernel_14340\981086241.py:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~i

면접관의 피드백: 피드백: 답변이 너무 일반적이고 구체적인 내용을 제공하지 않습니다. 좀 더 구체적으로 어떤 부분을 공부하겠다는 것인지 설명해주거나, 어떤 자원을 활용할 것인지 언급하는 것이 도움이 될 것입니다. 또한, 질문에 대한 직접적인 답변이 아니므로 개선이 필요합니다.


KeyboardInterrupt: 